In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from dateutil import relativedelta
import random

from sklearn.preprocessing import MinMaxScaler

from backtesting import Backtest, Strategy

import torch
import torch.nn as nn
from graphtorch import SparseMatrix, SparseModel

## quarter

- 01 : 01~03
- 02 : 04~06
- 03 : 07~09
- 04 : 10~12

In [3]:
def to_datetime_debug(x) : 
    try :
        return pd.to_datetime(x, format='%Y%m%d%H')
    except :
        adjusted = str(x)
        last = str(x)[-2:]    
        if last == '24' : 
            adjusted = adjusted[:-2] + '00'
        temp = pd.to_datetime(adjusted, format='%Y%m%d%H')
        return temp

In [4]:
def get_dust_by_year_quarter(year, quarter) : 
    
    filepath = 'data/finedust/raw_pickle/%s/%s-%.2d.pkl'%(year, year, quarter)
    #print(filepath)
    df = pd.read_pickle(filepath)
    #display(df)
    if (year>=2016) and (quarter >=3) : 
        df = df[(df['지역']=='서울 중구')&(df['측정소명']=='중구')]
    else : 
        df = df[(df['지역']=='서울')&(df['측정소명']=='중구')]
    df.index = df['측정일시'].apply(lambda x : to_datetime_debug(x))
    df = df.drop(['측정일시', '주소', '지역', '측정소코드', '측정소명'], axis=1)
    #display(df)
    return df
    
year = 2016
quarter = 1
df_test = get_dust_by_year_quarter(year, quarter)

In [5]:
ticker = '126880'
def get_data_by_ticker(ticker) : 
    
    # 1. 주식데이터
    filepath_stock = 'data/stock/%s.csv'%ticker
    df_stock = pd.read_csv(filepath_stock, index_col=0)
    df_stock.index = pd.to_datetime(df_stock.index, format='%Y%m%d')
    if 'Adj Close' in df_stock.columns :
        df_stock = df_stock.drop(['Adj Close'], axis=1)
    #display(df)
    
    # 2. 먼지 데이터
    datetime_first = datetime.datetime(year=df_stock.index[0].year, 
                              month=df_stock.index[0].month,
                              day=1)
    datetime_last = datetime.datetime(year=df_stock.index[-1].year,
                            month=df_stock.index[-1].month,
                            day=1)
    
    datetime_dust = []
    current_datetime = datetime_first
    while current_datetime != datetime_last :
        datetime_dust.append(current_datetime)
        current_datetime += relativedelta.relativedelta(months=1)
    df_dust = None
    quarter = 1
    for idx, corresponding_month in enumerate(datetime_dust) :
        year = corresponding_month.year
        month = corresponding_month.month
        
        new_quarter = ( (month-1)//3 ) + 1
        if idx == 0 :
            df_dust = get_dust_by_year_quarter(year, quarter)
           
        elif new_quarter != quarter : 
            df_dust = pd.concat([df_dust, get_dust_by_year_quarter(year, quarter)], axis=0)
            quarter = new_quarter   
    
    return df_stock, df_dust

df_stock, df_dust = get_data_by_ticker(ticker)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.6/

In [6]:
df_stock

,High,Low,Open,Close,Volume
2011-01-31,9760,8509,8509,9761,1781870
2011-02-01,9586,8301,9447,8510,3166897
2011-02-07,8787,7745,8683,7781,869448
2011-02-08,8336,7502,7884,7781,681818
2011-02-09,8023,7711,7884,7850,399132
...,...,...,...,...,...
2019-10-29,5470,5320,5420,5370,165524
2019-10-30,5500,5260,5410,5320,337981
2019-10-31,5510,5320,5350,5440,301572
2019-11-01,5640,5420,5470,5530,518471


In [7]:
df_dust

,CO,NO2,O3,PM10,PM25,SO2,망
측정일시,,,,,,,
2011-01-01 01:00:00,0.7,0.022,0.007,46,NaN,0.008,NaN
2011-01-01 02:00:00,0.8,0.021,0.007,46,NaN,0.008,NaN
2011-01-01 03:00:00,0.7,0.02,0.009,47,NaN,0.008,NaN
2011-01-01 04:00:00,0.8,0.021,0.007,44,NaN,0.007,NaN
2011-01-01 05:00:00,0.7,0.014,0.014,47,NaN,0.008,NaN
...,...,...,...,...,...,...,...
2019-03-31 20:00:00,0.3,0.019,0.041,22,11,0.002,도시대기
2019-03-31 21:00:00,0.3,0.014,0.043,17,11,0.002,도시대기
2019-03-31 22:00:00,0.4,0.016,0.044,19,10,0.002,도시대기


In [8]:
def merge_stock_dust(df_stock, df_dust) :
    df = df_stock.merge(df_dust, how='inner', left_on=df_stock.index, right_on=df_dust.index)
    df.index = df['key_0']
    df = df.drop(['key_0', '망', 'PM25'], axis=1)
    df = df.dropna(axis=0)
    for column in df.columns :
        df[column] = df[column].astype('float')
    return df

df = merge_stock_dust(df_stock, df_dust)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1575 entries, 2011-01-31 to 2019-04-30
Data columns (total 10 columns):
High      1575 non-null float64
Low       1575 non-null float64
Open      1575 non-null float64
Close     1575 non-null float64
Volume    1575 non-null float64
CO        1575 non-null float64
NO2       1575 non-null float64
O3        1575 non-null float64
PM10      1575 non-null float64
SO2       1575 non-null float64
dtypes: float64(10)
memory usage: 135.4 KB


In [10]:
df

,High,Low,Open,Close,Volume,CO,NO2,O3,PM10,SO2
key_0,,,,,,,,,,
2011-01-31,9760.0,8509.0,8509.0,9761.0,1781870.0,2.0,0.046,0.003,61.0,0.008
2011-01-31,9760.0,8509.0,8509.0,9761.0,1781870.0,2.0,0.046,0.003,61.0,0.008
2011-02-01,9586.0,8301.0,9447.0,8510.0,3166897.0,1.2,0.045,0.003,80.0,0.008
2011-02-01,9586.0,8301.0,9447.0,8510.0,3166897.0,1.2,0.045,0.003,80.0,0.008
2011-02-07,8787.0,7745.0,8683.0,7781.0,869448.0,1.3,0.042,0.003,76.0,0.006
...,...,...,...,...,...,...,...,...,...,...
2019-04-24,6350.0,6090.0,6290.0,6120.0,514428.0,0.5,0.048,0.008,38.0,0.005
2019-04-25,6160.0,5950.0,6120.0,6010.0,285335.0,0.4,0.024,0.023,15.0,0.003
2019-04-26,6120.0,5800.0,6030.0,5860.0,407115.0,0.3,0.021,0.032,8.0,0.002


In [11]:
def split_train_test(df) : 
    df_train = df[:'2018-10-1']
    df_valid = df['2018-10-1':]
    return df_train, df_valid
df_train, df_valid = split_train_test(df)

In [12]:
df_train.head()

,High,Low,Open,Close,Volume,CO,NO2,O3,PM10,SO2
key_0,,,,,,,,,,
2011-01-31,9760.0,8509.0,8509.0,9761.0,1781870.0,2.0,0.046,0.003,61.0,0.008
2011-01-31,9760.0,8509.0,8509.0,9761.0,1781870.0,2.0,0.046,0.003,61.0,0.008
2011-02-01,9586.0,8301.0,9447.0,8510.0,3166897.0,1.2,0.045,0.003,80.0,0.008
2011-02-01,9586.0,8301.0,9447.0,8510.0,3166897.0,1.2,0.045,0.003,80.0,0.008
2011-02-07,8787.0,7745.0,8683.0,7781.0,869448.0,1.3,0.042,0.003,76.0,0.006


In [13]:
df_valid.head()

,High,Low,Open,Close,Volume,CO,NO2,O3,PM10,SO2
key_0,,,,,,,,,,
2018-10-01,3830.0,3710.0,3785.0,3785.0,106970.0,0.4,0.012,0.031,17.0,0.002
2018-10-02,3840.0,3610.0,3775.0,3650.0,123484.0,0.5,0.049,0.002,21.0,0.003
2018-10-10,3830.0,3640.0,3740.0,3655.0,109651.0,0.3,0.012,0.024,13.0,0.002
2018-10-11,3595.0,3380.0,3450.0,3380.0,171034.0,0.4,0.029,0.009,18.0,0.002
2018-10-12,3490.0,3250.0,3250.0,3410.0,229214.0,0.7,0.051,0.002,28.0,0.003


In [87]:
def scale_data(df_train, df_valid) : 
    scaler = MinMaxScaler()
    for column in df_train.columns : 
        df_train[column] = scaler.fit_transform(df_train[column].values.reshape(-1, 1))
        df_valid[column] = scaler.transform(df_valid[column].values.reshape(-1, 1))
    return df_train, df_valid
df_train, df_valid = scale_data(df_train, df_valid)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [88]:
df_train.to_pickle('df_train.pkl')
df_valid.to_pickle('df_valid.pkl')

In [89]:
df_train = pd.read_pickle('df_train.pkl')
df_valid = pd.read_pickle('df_valid.pkl')

In [90]:
def create_first_generation(in_dim, out_dim, p, num_of_children) :
    assert (p<=1)
    assert (p>0)
    # p : possibility of connection
    
    children_matrix = []
    for idx_child in range(0, num_of_children) : 
        # create combinations
        combinations = []
        for idx_in in range(0, in_dim) :
            for idx_out in range(0, out_dim) : 
                combinations.append((idx_in, idx_out))

        # create connection with possibility p
        connections = []
        for combination in combinations :
            connection = random.choices([0, 1], weights=[1-p,p])
            if connection[0] == 1 :
                connections.append(combination)

        # create matrix without connection
        mat = np.zeros((out_dim, in_dim))

        # connect with list connections
        for connection in connections :
            mat[connection[1], connection[0]] = 1

        # wrap with SparseMatrix class
        child_matrix = SparseMatrix(mat, in_dim, out_dim)
        
        # append to children
        children_matrix.append(child_matrix)
    
    return children_matrix

in_dim = df_train.shape[1]
out_dim = 3
children_matrix = create_first_generation(in_dim, out_dim, 0.2, 5)

In [91]:
children_matrix[0].mat

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [92]:
children_matrix[1].mat

array([[0., 1., 0., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 1., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [93]:
children_matrix[2].mat

array([[0., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [94]:
children_matrix[3].mat

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 0., 0., 1., 0., 0.]])

In [95]:
children_matrix[4].mat

array([[1., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [1., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0., 0.]])

In [120]:
class WANNknowsALL(Strategy) :
    def init(self) :    
        self.close = self.data['Close']
        self.signal_buy = self.data['signal_buy']
        self.signal_hold = self.data['signal_hold']
        self.signal_sell = self.data['signal_sell']
        
    def next(self) : 
        if (self.signal_buy > max(self.signal_sell, self.signal_hold)):
            self.buy()

        if (self.signal_sell > max(self.signal_buy, self.signal_hold)) :
            self.sell()

In [127]:
def get_fitness(sparsemat, data, activations) : 
    
    df = data.copy()
    data = torch.from_numpy(data.values).float()
    score = []
    
    for constant_weight in [-2, -1, -0.5, 0.5, 1, 2] :
        model = SparseModel(sparsemat, activations, constant_weight, nn.Softmax())
        model.eval()
        yhat, nodes = model(data)
        yhat = yhat.detach().numpy()
        df['signal_buy'] = yhat[:, 0]
        df['signal_hold'] = yhat[:, 1]
        df['signal_sell'] = yhat[:, 2]
        #display(df)
        
        try : 
            bt = Backtest(df, WANNknowsALL, cash=10000)
            result = bt.run()
            #print(result)
            #bt.plot()
            exposure = result[3]
            backtest_return = result[6] # return을 fitness 함수로 사용. maximize
            # exposure가 한번도 일어나지 않았을 경우 (거래가 일어나지 않았을 경우 페널티를 줌)
            if exposure == 0 :
                backtest_return = -1000
            score.append(backtest_return)
        except : # 이유는 모르겠는데 backtesting 자체에 에러를 내는 경우가 있음.
            # 이 경우에도 exposure가 일어나지 않은 것과 동일한 페널티
            score.append(-1000)
    
    return sum(score)/len(score) # return mean score

activations = [None, None, nn.ReLU(), nn.Sigmoid()]
get_fitness(children_matrix[0], df_train, activations)

-550.0

In [141]:
activations = activations = [None, None, nn.ReLU(), nn.Sigmoid()]

def NEAT(df_train, df_valid, activations, p=0.2, num_children_in_generation=5) : 
    
    # 1. 첫번째 Generation 생성
    children_matrix = create_first_generation(df_train, p, num_children_in_generation)
    children_score_train = [get_fitness(x, df_train, activations) for x in children_matrix]
    children_score_valid = [get_fitness(x, df_valid, activations) for x in children_matrix]
    ## train score가 높은 순으로 정렬
    df = pd.DataFrame(columns=['matrix','score_train','score_valid'])
    df['matrix'] = children_matrix
    df['score_train'] = children_score_train
    df['score_valid'] = children_score_valid
    df = df.sort_values(by='score_train', ascending=False)
    ## rank-base sorted matrix for next generation
    candidates_matrix = df['matrix']
    
    mean_score_train = df['score_train'].mean()
    mean_score_valid = df['score_valid'].mean()
    
    
    # 2. valid score가 감소하기 시작할때까지 (train에 ovefit 될떄까지) 반복
    current_mean_score_valid = mean_score_valid +1 # 첫 loop를 들어가기 위한 초기화
    while current_mean_score_valid > mean_score_valid : # current_mean_score가 이전 step보다 더 작으면 종료
        
        ## a) mean_score_valid에 이전 generation의 mean valid score입력
        mean_score_valid = current_mean_score_valid
        
        ## b) candidates_matrix에서 상위 n개 개체에 대해 random하게 3가지의 operation중 하나를 수행하여 다시 candidates_matrix에 담음

        ## c) 각각의 candidates_matrix를 get_fitness로 train, valid set에 대해 평가
        ##     df에 matrix, score_train, score_valid 담음

        ## d) score_train을 기준으로 sort

        ## e) candidates_matrix를 sort된 순서대로(score가 높은 순서대로) 업데이트
        
        ## f) 현재 gen의 mean score valid 계산
        current_mean_score_valid = df['score_valid'].mean()
    
    return df
    
NEAT(df_train, df_valid, activations, 0.4, 5)

,matrix,score_train,score_valid
2,<graphtorch.SparseMatrix object at 0x7f7a2b1de...,-550.0,-100.0
0,<graphtorch.SparseMatrix object at 0x7f7a2b25b...,-550.0,-550.0
1,<graphtorch.SparseMatrix object at 0x7f7a2b1fa...,-550.0,-550.0
3,<graphtorch.SparseMatrix object at 0x7f7a2b1de...,-550.0,-550.0
4,<graphtorch.SparseMatrix object at 0x7f7a2b1de...,-550.0,-550.0
